# Дообучение GPT-2 для генерации текстов песен метал групп.

В этом ноутбуке представлен fine-tuning модели GPT-2 для генерации тесктов песен метал групп на английском языке. Для работы с моделью GPT-2 используется библиотека Hugging Face Transformers.


## Подготовка

In [ ]:
from tqdm import tqdm

import pandas as pd
import numpy as np
import random

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup

In [2]:
# если ноутбук запускается в колабе, лучше подключить google drive для сохранения весов модели
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Задаем параметры обучения

In [ ]:
args = {
    "batch_size": 2,        # размер батча (размер 2 был выбран, так как при большем размере память GPU переполняется)
    "train_size": 0.8,      # доля тренеровочной выборки от всего датасета
    "epochs": 5,            # количество эпох обучения
    "learning_rate": 5e-4,  # темп обучения модели
    "warmup_steps": 100,    # количество шагов, после которого шедуллер будет уменьшать learning rate
}

Загружаем датасет

In [ ]:
data = pd.read_csv("dataset.csv")["lyrics"]
data

,lyrics
0,Leave me alone\nLike a dog with a bone\nLike a...
1,"""The following is a true story, only the names..."
2,"""Get ready\nYou think your kinda tough\nYou're..."
3,Do you wanna boogie?\nDo you wanna blow?\nShe ...
4,"Got a taste of a rocking band,\nStanding there..."
...,...
10541,I make a call\nSo far to fall\nRestless cravin...
10542,Periodic tableware\nPsychotropic science fare\...
10543,The 28th day\nShe'll be bleeding again\nAnd in...
10544,She thinks I'm iron man that I don't feel pain...


In [ ]:
data = data.sample(len(data))

In [4]:
# max_len = max([len(text) for text in data])
max_len = 6290
print("максимальная длина текста в датасете:", max_len)

максимальная длина текста в датасете: 6290


Загружаем токенизатор для GPT-2

In [5]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2', bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>', max_length=max_len)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Описываем класс датасета

In [ ]:
class GPT2Dataset(Dataset):

  def __init__(self, txt_list, tokenizer, max_length=768):

    self.tokenizer = tokenizer
    self.input_ids = []
    self.attn_masks = []

    # токенизируем текста
    for txt in txt_list:

      encodings_dict = tokenizer('<|startoftext|>'+ txt + '<|endoftext|>', truncation=True, max_length=max_length, padding="max_length")

      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx]

Преобразуем датасет из `pandas.Series` в наш класс и разделяем выборку на обучающую и валидационную

In [ ]:
dataset = GPT2Dataset(data, tokenizer, max_length=max_len)

train_size = int(len(dataset) * args["train_size"])
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print("train size:", len(train_dataset))
print("validation size:", len(val_dataset))

train size: 8436
validation size: 2110


In [ ]:
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), # Перемешиваем семплы после каждой эпохи
            batch_size = args["batch_size"]
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset), # Достаем семплы последовательно
            batch_size = args["batch_size"]
        )

In [6]:
# инициализируем предобученную модель
configuration = GPT2Config.from_pretrained('gpt2', output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained("gpt2", config=configuration)

# переопределяем токенизатор для модели, так как ранее мы добавили новые токены
model.resize_token_embeddings(len(tokenizer))

# записываем в device устройство, на котором будем обучать модель и загружаем туда саму модель
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# для воспроизводимости определим random seed для всех рандомайзеров в библиотеках
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


В качестве оптимизатора выберем `AdamW` из библиотеки Hugging Face

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = args["learning_rate"],
                  eps = 1e-8,
                )

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Выберем линейный шедулер, который сначала увеличивает learning rate от 0 до заданного в словаре `args`, потом после `num_warmup_steps` шагов линейно уменьшает lr обратно до 0

In [ ]:
total_steps = len(train_dataloader) * args["epochs"]

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = args["warmup_steps"],
                                            num_training_steps = total_steps)

## Цикл обучения модели

В этом цикле во время каждой эпохи модель сначала обучается, затем валидируется. Процесс обучения во во время отдельной эпохи логируется в консоли при помощи библиотеки `tqdm`. После каждой эпохи выводится средние лоссы на трейне и валидации. Если на текущей эпохе лосс на валидации оказался меньше всех предыдущих, веса модели сохраняются на google диск. После всего процесса обучения лоссы на трейне и валидации запишутся в массивы `train_loss` и `val_loss` соответственно.

In [ ]:
train_loss = []
val_loss = []

best_loss = float("inf")

model.to(device)

for epoch_i in range(3):

    total_train_loss = 0

    model.train()
    train_loop = tqdm(train_dataloader, leave=False)

	# обучаем модель 
    for text, mask in train_loop:

        b_input_ids = text.to(device)
        b_labels = text.to(device)
        b_masks = mask.to(device)

        model.zero_grad()

        outputs = model(  b_input_ids,
                          labels=b_labels,
                          attention_mask = b_masks,
                          token_type_ids=None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        loss.backward()

        optimizer.step()
        scheduler.step()

	# лосс на тренеровочной выборке на текущей эпохе 
    avg_train_loss = total_train_loss / len(train_dataloader)

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # валидируем модель
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(b_input_ids,
							# token_type_ids=None,
                            attention_mask = b_masks,
                            labels=b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

	# лосс на валидации на текущей эпохе 
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        torch.save(model.state_dict(), "drive/MyDrive/ml/model_state_dict.pt")

    print(f'Epoch {epoch_i + 1}/{args["epochs"]}, train loss {avg_train_loss:.4f}, val loss {avg_val_loss:.4f}')

    # добавляем лоссы на текущей эпохе в массивы 
    train_loss.append(avg_train_loss)
    val_loss.append(avg_val_loss)

Epoch 1/5, train loss 1.1170, val loss 1.0577


Epoch 2/5, train loss 0.9875, val loss 1.0400


Epoch 3/5, train loss 0.8637, val loss 1.0407


## Генерация текста

Загрузим в модель веса, с которыми она показала лучшие результаты на валидации и напишем функцию для генерации текста вручную. Это потребовалось, так как при использовании готового метода `generate` модель зацикливается даже при большой температуре, большом `top_k` и штрафе за повторения.

In [23]:
model.load_state_dict(torch.load("drive/MyDrive/ml/model_state_dict.pt", map_location = "cpu"))

def my_generate(context = "", length = 300, temperature = 1.0) -> str:
  '''
  context - начало текста
  length - длина итогового текста в токенах, считая context
  temperature - температура

  Возвращает строку со сгенерированным текстом
  '''

  context = "<|startoftext|>" + context
  context = tokenizer.encode(context, add_special_tokens=False, return_tensors="pt")

  model.to("cpu")
  model.eval()
  # генерируем текст
  with torch.no_grad():
    for _ in range(length - len(context)):
      outputs = model(context)[0][:, -1, :]  # предсказываем вероятности токенов
      next_token_logits = outputs / (temperature if temperature > 0 else 1.) # применяем температуру
      next_token = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1) # определяем следующий токен
      context = torch.cat((context, next_token), dim=1) # добавляем токен к последовательности

  # декодируем токены в строку и обрезаем ее от токена начала до токена конца, если он есть
  text = tokenizer.decode(context[0], clean_up_tokenization_spaces=True)[len("<|startoftext|>"):]
  end = text.find("<|endoftext|>")

  if end != -1:
    text = text[:end]

  return text

# пробуем сгенерировать продолжение песни Master of Puppets по первому куплету
print("\n-------Generated Text-------")
print(my_generate("End of passion play, crumbling away\n\
I'm your source of self-destruction\n\
Veins that pump with fear, sucking darkest clear\n\
Leading on your death's construction\n"))

<ipython-input-23-f05ffe0b3403>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/drive/MyDrive/ml/song_model_loop.pt", map_locatio


-------Generated Text-------
End of passion play, crumbling away
I'm your source of self-destruction
Veins that pump with fear, sucking darkest clear
Leading on your death's construction
Eternally condemned to life you're buried alive
From a world without a memory
Manipulate, disguise, priest, liar
of the brainrist
Destroyer of the lost throne of fear, hanging by a thread
Bathed in blood, i crown thee
Who's dedicated to this in search for flesh
In desensitized greed and pain
Writhing ray of tortured victimize
Contummate, captive

Total escape and decay
The end has come
The hunger vultures wait the world falls down by its feet

Blessed are slaughtered
Seventh- Capricorn smiles
Blood now drips from your skull
Torn apart
Body devastation
Commit to lust

Instead of f*cking co-dependent
The worst place on earth
The judicial system
That is pure hate is accomplished
In God's creation
The end has come
The hunger vultures wait the world falls down by its feet

Blessed are slaughtered
Seventh- 

In [25]:
# теперь попробуем сгенерировать песню с нуля
print(my_generate())

Sometimes it seems everything's done you're leaving behind
I thought they had something to say
But that wouldn't change my mind
They said nothing but dysfunction
I know you all have lost their confidence in the world
Call it now before the end
Stay out of your life
Please stay out of enemy and friend
'Cause I'm here at last 
Forgive pain, no regrets 
But make up your mind
Carving the road to nowhere you said 
Check the sign of the times at which you're facing the obstacles 
Mercy in that survival is possible, well 

It's time to rise up and defeat the tides of this wreckin' on you 
And I don't believe there's a need then you've just begun and questioned your machine
I'm bangin' for speed I'm maimin' for seizure, I'm An expert in Dealin'.

Let go
I'll be here 'cause I'm here at last 
I'll give you now what you left, it's time to stand up and protect yourself 
The young ideas that make it through flight 
 patients who miracle when you're paralyzed 
Are not enough to run away
Och don't tu